# **Modelo GNN para Ordenamiento de Nodos**

Implementación de una Red Neuronal de Grafos (GNN) para aprender ordenamientos óptimos de nodos.

In [ ]:
%run librerias.ipynb

## **1. ARQUITECTURA GNN**

In [ ]:
class GNNOrdering(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=2):
        super().__init__()
        self.convs = nn.ModuleList()

        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))

        self.lin = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.3, training=self.training)

        scores = self.lin(x).squeeze()
        return scores

## **2. FUNCIÓN DE COLORACIÓN GREEDY PARA GNN**

In [ ]:
def greedy_coloring_gnn(edge_index, num_nodes, ordering):
    color = [-1] * num_nodes
    adj = [[] for _ in range(num_nodes)]

    edges = edge_index.cpu().numpy()
    for u, v in edges.T:
        adj[u].append(v)
        adj[v].append(u)

    for v in ordering:
        used = set(color[u] for u in adj[v] if color[u] != -1)
        c = 0
        while c in used:
            c += 1
        color[v] = c

    return max(color) + 1

## **3. PREPARACIÓN DE DATOS PARA GNN**

In [ ]:
def preparar_datos_gnn(num_nodes=1000, num_edges=300):
    """
    Crea un grafo sintético para entrenamiento de GNN
    """
    edge_index = torch.randint(0, num_nodes, (2, num_edges))

    deg = torch.zeros(num_nodes, 1)
    for u, v in edge_index.t():
        deg[u] += 1
        deg[v] += 1

    data = Data(
        x=deg,
        edge_index=edge_index,
        num_nodes=num_nodes
    ).to(device)
    
    return data

## **4. INICIALIZACIÓN DEL MODELO**

In [ ]:
model = GNNOrdering(
    in_channels=1,
    hidden_channels=32,
    num_layers=2
).to(device)

print("Modelo GNN inicializado:")
print(model)